# Tensor Operations with Tensorflow


In [1]:
import tensorflow as tf
import numpy as np

## Tensor Product & Square & Sum

In [2]:

a = tf.placeholder(tf.float32, shape=(1, 3))
b = tf.placeholder(tf.float32, shape=(1, 2))
c = tf.placeholder(tf.float32, shape=(1, 3))
# two vectors outer dot -> a and b. 
ab = tf.tensordot(a, b, axes=([0], [0]))
# reshape the ab matrix into a 3D tensor
ab_ = tf.reshape(ab, shape=(3 ,2, 1))
# 2D matrix outer dot with the third vector
abc = tf.tensordot(ab_, c, axes=([2], [0])) 

# test the square for each element of vector
a_2 = tf.square(a)
# test the square for each element of matrix
ab_2 = tf.square(ab)
# calculate the sum of squared matrix
ab_2sum = tf.reduce_sum(ab_2, reduction_indices=[0,1])
# product the matrix with a constant
lamda = tf.Variable(2.0, tf.float32)
ab_lamda = lamda*ab

with tf.Session() as session:
    # initialize the variables
    session.run(tf.global_variables_initializer())
    # print ab
    print ((ab.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]})))
    # print ab_
    print ((ab_.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]})))
    # print abc
    X = abc.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]}
    )
    # print ab_2
    print ab_2.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})
    # print ab_2sum
    print ab_2sum.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})
    # print ab_lamda
    print ab_lamda.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})



[[  5.   6.]
 [ 10.  12.]
 [ 15.  18.]]
[[[  5.]
  [  6.]]

 [[ 10.]
  [ 12.]]

 [[ 15.]
  [ 18.]]]
[[  25.   36.]
 [ 100.  144.]
 [ 225.  324.]]
854.0
[[ 10.  12.]
 [ 20.  24.]
 [ 30.  36.]]


## Filter the NAN Value

In [3]:
A = [1,1,np.nan]

In [4]:
x = tf.placeholder(tf.float64, shape=[None, None])
nanX = tf.is_nan(x)
y = tf.boolean_mask(x,tf.logical_not(nanX),name='boolean_mask')
with tf.Session() as session:
    print ((nanX.eval(
        feed_dict={ x: [[np.nan,6]]})))
    print (y.eval(feed_dict={x:[[np.nan, 6],[4, np.nan]]}))

[[ True False]]
[ 6.  4.]
